In [1]:
# Uncomment to install/upgrade ras-commander from pip
#!pip install --upgrade ras-commander

#Import the ras-commander package
from ras_commander import *

##### Optional Code Cell For Development/Testing Mode (Local Copy)
##### Uncomment and run this cell instead of the pip cell above

```python
# For Development Mode, add the parent directory to the Python path
import os
import sys
from pathlib import Path

current_file = Path(os.getcwd()).resolve()
rascmdr_directory = current_file.parent

# Use insert(0) instead of append() to give highest priority to local version
if str(rascmdr_directory) not in sys.path:
    sys.path.insert(0, str(rascmdr_directory))

print("Loading ras-commander from local dev copy")
from ras_commander import *
```

# HEC-RAS Steady State Flow Analysis

This notebook demonstrates how to extract and analyze steady state flow results from HEC-RAS using the ras-commander library. It showcases the new steady state functionality in `HdfResultsPlan`.

## New Steady State Methods

The library now includes full support for steady state analysis:
- `is_steady_plan()` - Check if HDF contains steady state results
- `get_steady_profile_names()` - Extract steady state profile names
- `get_steady_wse()` - Extract water surface elevations for profiles
- `get_steady_info()` - Extract steady flow metadata and attributes

## Extracting Steady Flow Computation Messages

For steady flow analyses, computation messages provide valuable information about:
- Hydraulic computations and convergence
- Warning messages for critical flow or other conditions
- Computation timing and performance

We can extract these using `HdfResultsPlan.get_compute_messages()`:

In [11]:
bald_eagle_path = RasExamples.extract_project("Balde Eagle Creek")

2025-11-17 15:14:12 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2025-11-17 15:14:12 - ras_commander.RasExamples - INFO - Extracting project 'Balde Eagle Creek'
2025-11-17 15:14:12 - ras_commander.RasExamples - INFO - Project 'Balde Eagle Creek' already exists. Deleting existing folder...
2025-11-17 15:14:12 - ras_commander.RasExamples - INFO - Existing folder for project 'Balde Eagle Creek' has been deleted.
2025-11-17 15:14:12 - ras_commander.RasExamples - INFO - Successfully extracted project 'Balde Eagle Creek' to c:\GH\ras-commander\examples\example_projects\Balde Eagle Creek


In [8]:
plan_number = "01"

In [20]:
init_ras_project(bald_eagle_path, "6.6")

2025-11-17 15:17:06 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.rasmap


['<ras_commander.RasPrj.RasPrj at 0x282e0dc5550>']

In [21]:
ras.plan_df

['  plan_number unsteady_number geometry_number                     Plan Title  \\\n', '0          01              02              01  Unsteady with Bridges and Dam   \n', '1          02            None              01                Steady Flow Run   \n', '\n', '  Program Version Short Identifier                  Simulation Date  \\\n', '0            5.00     UnsteadyFlow    18FEB1999,0000,24FEB1999,0500   \n', '1             NaN        SteadyRun  02/18/1999,0000,02/24/1999,0500   \n', '\n', '  Computation Interval Mapping Interval Run HTab  ... PS Cores DSS File  \\\n', '0                 2MIN            1HOUR        1  ...     None      dss   \n', '1                 2MIN              NaN        1  ...     None      dss   \n', '\n', '  Friction Slope Method                                   HDF_Results_Path  \\\n', '0                     2  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '1                     1                                               None   \n', '\n', '  Geom File                                          Geom Path  Flow File  \\\n', '0        01  C:\\GH\\ras-commander\\examples\\example_projects\\...         02   \n', '1        01  C:\\GH\\ras-commander\\examples\\example_projects\\...         02   \n', '\n', '                                           Flow Path  \\\n', '0  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '1  C:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '\n', '                                           full_path flow_type  \n', '0  C:\\GH\\ras-commander\\examples\\example_projects\\...  Unsteady  \n', '1  C:\\GH\\ras-commander\\examples\\example_projects\\...    Steady  \n', '\n', '[2 rows x 27 columns]']

In [18]:
RasCmdr.compute_plan(plan_number)

2025-11-17 15:15:21 - ras_commander.RasCmdr - INFO - Using ras_object with project folder: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek
2025-11-17 15:15:21 - ras_commander.RasCmdr - INFO - Running HEC-RAS from the Command Line:
2025-11-17 15:15:21 - ras_commander.RasCmdr - INFO - Running command: "C:\Program Files (x86)\HEC\HEC-RAS\6.6\Ras.exe" -c "C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.prj" "C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01"
2025-11-17 15:16:58 - ras_commander.RasCmdr - INFO - HEC-RAS execution completed for plan: 01
2025-11-17 15:16:58 - ras_commander.RasCmdr - INFO - Total run time for plan 01: 97.74 seconds


['True']

## Extracting Steady Flow Computation Messages

For steady flow analyses, computation messages provide valuable information about:
- Hydraulic computations and convergence
- Warning messages for critical flow or other conditions
- Computation timing and performance

We can extract these using `HdfResultsPlan.get_compute_messages()`:

In [24]:
# Extract computation messages for steady flow analysis
from ras_commander.HdfResultsPlan import HdfResultsPlan

print("="*80)
print("STEADY FLOW COMPUTATION MESSAGES")
print("="*80)

# Extract messages (works with plan number or HDF path)
steady_msgs = HdfResultsPlan.get_compute_messages(plan_number)

if steady_msgs:
    print(f"\nExtracted {len(steady_msgs)} characters\n")
    
    # Display messages
    print("Computation messages:")
    print("-" * 80)
    print(steady_msgs[:1000])  # First 1000 characters
    
    if len(steady_msgs) > 1000:
        print("\n... (truncated for display) ...")
    
    # Look for critical information
    print("\n" + "="*80)
    print("Checking for critical flow or warnings...")
    print("="*80)
    
    lines = steady_msgs.split('\n')
    critical = [l for l in lines if 'critical' in l.lower() or 'warning' in l.lower()]
    
    if critical:
        print(f"Found {len(critical)} lines with critical flow or warnings:")
        for line in critical[:10]:
            print(f"  - {line.strip()}")
    else:
        print("✓ No critical flow or warning messages found")
else:
    print("No computation messages available")

print("\n" + "="*80)

2025-11-17 15:18:45 - ras_commander.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01.hdf
2025-11-17 15:18:45 - ras_commander.HdfResultsPlan - INFO - Reading computation messages from HDF: BaldEagle.p01.hdf
2025-11-17 15:18:45 - ras_commander.HdfResultsPlan - INFO - Successfully extracted 1693 characters from HDF


STEADY FLOW COMPUTATION MESSAGES

Extracted 1693 characters

Computation messages:
--------------------------------------------------------------------------------
Plan: 'Unsteady with Bridges and Dam' (BaldEagle.p01)
Simulation started at: 17Nov2025 03:15:22 PM

Writing Plan GIS Data...
Completed Writing Plan GIS Data
Writing Geometry...
Computing Bank Lines
Bank lines generated in 119 ms
Computing Edge Lines
Edge Lines generated in 51 ms
Computing XS Interpolation Surface
XS Interpolation Surface generated in 122 ms
Completed Writing Geometry
Writing Event Conditions ...
Completed Writing Event Condition Data

	
Geometric Preprocessor HEC-RAS 6.6 September 2024
 

Finished Processing Geometry


Performing Unsteady Flow Simulation  HEC-RAS 6.6 September 2024
 
	
Unsteady Input Summary:
     1D Unsteady Finite Difference Numerical Solution

Overall Volume Accounting Error in Acre Feet:    -29.5468461514
Overall Volume Accounting Error as percentage:           0.01407
Please review "Com

## Package Installation and Environment Setup

In [25]:
# Install ras-commander from pip (uncomment to install if needed)
# !pip install --upgrade ras-commander

# Set to False to disable plot generation for llm-friendly outputs
generate_plots = True

# Enable this cell for local development version of ras-commander
import os
import sys      
from pathlib import Path
current_file = Path(os.getcwd()).resolve()
rascmdr_directory = current_file.parent
sys.path.append(str(rascmdr_directory))
print("Loading ras-commander from local dev copy")

# Import RAS-Commander modules
from ras_commander import *

In [26]:
# Import required modules
from ras_commander import *

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Set pandas display options
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

## Load Bald Eagle Creek Example Project

This project contains both unsteady (Plan 01) and **steady state** (Plan 02) flow analyses.

In [27]:
# Extract and initialize the Bald Eagle Creek project
current_dir = Path.cwd()
bald_eagle_path = current_dir / "example_projects" / "Balde Eagle Creek"

# Extract project if needed
if not bald_eagle_path.exists():
    RasExamples.extract_project("Balde Eagle Creek")
    print("Extracted Bald Eagle Creek project")
else:
    print("Bald Eagle Creek project already exists")

# Initialize the project
init_ras_project(bald_eagle_path, "6.6")
print(f"Initialized project: {ras.project_name}")

2025-11-17 15:18:45 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.rasmap


Bald Eagle Creek project already exists
Initialized project: BaldEagle


In [28]:
# View all plans in the project
print("Plans in this project:")
ras.plan_df[['plan_number', 'Plan Title', 'unsteady_number', 'Program Version']]

Plans in this project:


['  plan_number                     Plan Title unsteady_number Program Version\n', '0          01  Unsteady with Bridges and Dam              02            5.00\n', '1          02                Steady Flow Run            None             NaN']

## Run Steady State Plan (Plan 02)

Execute the steady state plan if results don't already exist.

In [29]:
# Define plan number
plan_number = "02"

# Check if results exist
plan02_hdf = bald_eagle_path / "BaldEagle.p02.hdf"

if not plan02_hdf.exists():
    print(f"Running Plan {plan_number} (Steady State)...")
    success = RasCmdr.compute_plan(plan_number)
    if success:
        print(f"Plan {plan_number} executed successfully")
    else:
        print(f"Plan {plan_number} execution failed")
else:
    print(f"Plan {plan_number} results already exist")

2025-11-17 15:18:45 - ras_commander.RasCmdr - INFO - Using ras_object with project folder: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek
2025-11-17 15:18:45 - ras_commander.RasCmdr - INFO - Running HEC-RAS from the Command Line:
2025-11-17 15:18:45 - ras_commander.RasCmdr - INFO - Running command: "C:\Program Files (x86)\HEC\HEC-RAS\6.6\Ras.exe" -c "C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.prj" "C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p02"


Running Plan 02 (Steady State)...


2025-11-17 15:18:50 - ras_commander.RasCmdr - INFO - HEC-RAS execution completed for plan: 02
2025-11-17 15:18:50 - ras_commander.RasCmdr - INFO - Total run time for plan 02: 4.96 seconds


Plan 02 executed successfully


## 1. Check if Plan Contains Steady State Results

Use `is_steady_plan()` to verify the HDF contains steady state results.

In [30]:
# Check if this is a steady state plan
is_steady = HdfResultsPlan.is_steady_plan(plan_number)
print(f"Is Plan {plan_number} a steady state plan? {is_steady}")

2025-11-17 15:18:50 - ras_commander.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p02.hdf


Is Plan 02 a steady state plan? True


## 2. Extract Steady State Profile Names

Get the list of all steady state profiles (e.g., different return periods).

In [31]:
# Get profile names
profiles = HdfResultsPlan.get_steady_profile_names(plan_number)

print(f"Found {len(profiles)} steady state profiles:")
for i, profile in enumerate(profiles, 1):
    print(f"  {i}. {profile}")

2025-11-17 15:18:50 - ras_commander.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p02.hdf
2025-11-17 15:18:50 - ras_commander.HdfResultsPlan - INFO - Found 8 steady state profiles: ['.5 year', '1 year', '2 year', '5 year', '10 year', '25 year', '50 year', '100 year']


Found 8 steady state profiles:
  1. .5 year
  2. 1 year
  3. 2 year
  4. 5 year
  5. 10 year
  6. 25 year
  7. 50 year
  8. 100 year


## 3. Extract Water Surface Elevations (WSE)

Extract WSE data for specific profiles or all profiles at once.

### 3a. Extract Single Profile by Name

In [32]:
# Extract WSE for 100-year profile
wse_100yr = HdfResultsPlan.get_steady_wse(plan_number, profile_name='100 year')

print(f"WSE Data for 100-year profile:")
print(f"Shape: {wse_100yr.shape}")
print(f"Columns: {list(wse_100yr.columns)}")
print("\nFirst 5 cross sections:")
wse_100yr.head()

2025-11-17 15:18:50 - ras_commander.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p02.hdf
2025-11-17 15:18:50 - ras_commander.HdfResultsPlan - INFO - Extracted WSE data for 1 profile(s), 178 cross sections


WSE Data for 100-year profile:
Shape: (178, 4)
Columns: ['River', 'Reach', 'Station', 'WSE']

First 5 cross sections:


['        River    Reach   Station         WSE\n', '0  Bald Eagle  Loc Hav  138154.4  669.521484\n', '1  Bald Eagle  Loc Hav  137690.8  669.346863\n', '2  Bald Eagle  Loc Hav  137327.0  668.883057\n', '3  Bald Eagle  Loc Hav  136564.9  666.177979\n', '4  Bald Eagle  Loc Hav  136202.3  666.057739']

### 3b. Extract Single Profile by Index

In [33]:
# Extract WSE for first profile (0.5-year) using index
wse_05yr = HdfResultsPlan.get_steady_wse(plan_number, profile_index=0)

print(f"WSE Data for {profiles[0]} profile:")
print(f"Shape: {wse_05yr.shape}")
print("\nSummary statistics:")
wse_05yr['WSE'].describe()

2025-11-17 15:18:50 - ras_commander.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p02.hdf
2025-11-17 15:18:50 - ras_commander.HdfResultsPlan - INFO - Extracted WSE data for 1 profile(s), 178 cross sections


WSE Data for .5 year profile:
Shape: (178, 4)

Summary statistics:


['count    178.000000\n', 'mean     590.215864\n', 'std       39.202902\n', 'min      537.500000\n', '25%      557.109406\n', '50%      579.655151\n', '75%      623.142242\n', 'max      660.588928\n', 'Name: WSE, dtype: float64']

### 3c. Extract All Profiles

In [34]:
# Extract WSE for all profiles
wse_all = HdfResultsPlan.get_steady_wse(plan_number)

print(f"WSE Data for all {len(profiles)} profiles:")
print(f"Shape: {wse_all.shape}")
print(f"Columns: {list(wse_all.columns)}")
print(f"\nProfiles included: {wse_all['Profile'].unique().tolist()}")
print("\nSample data:")
wse_all.head(10)

2025-11-17 15:18:50 - ras_commander.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p02.hdf
2025-11-17 15:18:50 - ras_commander.HdfResultsPlan - INFO - Extracted WSE data for 8 profile(s), 178 cross sections


WSE Data for all 8 profiles:
Shape: (1424, 5)
Columns: ['River', 'Reach', 'Station', 'Profile', 'WSE']

Profiles included: ['.5 year', '1 year', '2 year', '5 year', '10 year', '25 year', '50 year', '100 year']

Sample data:


['        River    Reach   Station  Profile         WSE\n', '0  Bald Eagle  Loc Hav  138154.4  .5 year  660.588928\n', '1  Bald Eagle  Loc Hav  137690.8  .5 year  659.914612\n', '2  Bald Eagle  Loc Hav  137327.0  .5 year  659.465759\n', '3  Bald Eagle  Loc Hav  136564.9  .5 year  658.126160\n', '4  Bald Eagle  Loc Hav  136202.3  .5 year  657.173157\n', '5  Bald Eagle  Loc Hav  135591.4  .5 year  656.520264\n', '6  Bald Eagle  Loc Hav  135068.7  .5 year  655.880676\n', '7  Bald Eagle  Loc Hav  134487.2  .5 year  654.593140\n', '8  Bald Eagle  Loc Hav  133881.0  .5 year  653.786621\n', '9  Bald Eagle  Loc Hav  133446.1  .5 year  653.006958']

## 4. Extract Steady Flow Metadata

Get plan information, program version, solution status, and flow file details.

In [35]:
# Get steady flow information
steady_info = HdfResultsPlan.get_steady_info(plan_number)

print(f"Steady Flow Information ({len(steady_info.columns)} attributes):")
print("\nKey attributes:")
for col in ['Program Version', 'Solution', 'Flow Title', 'Flow Filename']:
    if col in steady_info.columns:
        print(f"  {col}: {steady_info[col].values[0]}")

print("\nAll attributes:")
steady_info.T

2025-11-17 15:18:50 - ras_commander.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p02.hdf
2025-11-17 15:18:50 - ras_commander.HdfResultsPlan - INFO - Extracted 8 steady state attributes


Steady Flow Information (8 attributes):

Key attributes:
  Program Version: HEC-RAS 6.6 September 2024
  Solution: Steady Finished Successfully
  Flow Title: Steady Flow Data
  Flow Filename: BaldEagle.f02

All attributes:


['                                                                   0\n', 'Program Name                         HEC-RAS - River Analysis System\n', 'Program Version                           HEC-RAS 6.6 September 2024\n', 'Project File Name  C:\\GH\\ras-commander\\examples\\example_projects\\...\n', 'Type of Run                                     Steady Flow Analysis\n', 'Run Time Window             17NOV2025 15:18:48 to 17NOV2025 15:18:50\n', 'Solution                                Steady Finished Successfully\n', 'Flow Filename                                          BaldEagle.f02\n', 'Flow Title                                          Steady Flow Data']

## 5. Visualize Water Surface Profiles

Plot WSE vs. station for different return periods.

In [36]:
if generate_plots:
    # Create a plot comparing all profiles
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Plot each profile
    for profile in profiles:
        profile_data = wse_all[wse_all['Profile'] == profile]
        # Convert station to numeric for plotting
        stations = pd.to_numeric(profile_data['Station'], errors='coerce')
        ax.plot(stations, profile_data['WSE'], label=profile, linewidth=2)
    
    ax.set_xlabel('River Station (ft)', fontsize=12)
    ax.set_ylabel('Water Surface Elevation (ft)', fontsize=12)
    ax.set_title('Steady State Water Surface Profiles\nBald Eagle Creek', fontsize=14)
    ax.legend(title='Return Period', loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Invert X axis so upstream (higher stations) is on the left
    ax.invert_xaxis()
    
    plt.tight_layout()
    plt.show()
    
    # Print profile comparison stats
    print("\nProfile Comparison (Maximum WSE):")
    for profile in profiles:
        max_wse = wse_all[wse_all['Profile'] == profile]['WSE'].max()
        print(f"  {profile:10s}: {max_wse:.2f} ft")

<Figure size 1500x800 with 1 Axes>


Profile Comparison (Maximum WSE):
  .5 year   : 660.59 ft
  1 year    : 661.43 ft
  2 year    : 662.60 ft
  5 year    : 664.67 ft
  10 year   : 666.19 ft
  25 year   : 667.46 ft
  50 year   : 668.54 ft
  100 year  : 669.52 ft


## 6. Analyze WSE Differences Between Profiles

Compare water surface elevations between different return periods.

In [37]:
# Create a pivot table for easy comparison
wse_pivot = wse_all.pivot_table(
    index=['River', 'Reach', 'Station'],
    columns='Profile',
    values='WSE'
)

print("Water Surface Elevations by Profile and Station:")
print("\nFirst 10 stations:")
wse_pivot.head(10)

Water Surface Elevations by Profile and Station:

First 10 stations:


['Profile                         .5 year      1 year     10 year    100 year  \\\n', 'River      Reach   Station                                                    \n', 'Bald Eagle Loc Hav 100657.3  620.031982  657.029053  660.006348  662.170227   \n', '                   101440.3  620.033875  657.029053  660.006470  662.170593   \n', '                   10221.14  540.174683  541.939697  556.019287  563.945190   \n', '                   103122.3  620.044861  657.029053  660.006836  662.171753   \n', '                   103369.7  620.055420  657.029480  660.012878  662.186707   \n', '                   103854.0  620.059082  657.029480  660.012939  662.187012   \n', '                   104195.0  620.059875  657.029480  660.012878  662.186829   \n', '                   104647.2  620.066528  657.029480  660.013000  662.187378   \n', '                   105178.6  620.075806  657.029480  660.013367  662.188599   \n', '                   106466.0  620.093262  657.029419  660.013428  662.1887
...
[Output truncated, 2009 characters total]

In [38]:
# Calculate differences between profiles
if '100 year' in wse_pivot.columns and '.5 year' in wse_pivot.columns:
    wse_pivot['Diff_100yr_vs_05yr'] = wse_pivot['100 year'] - wse_pivot['.5 year']
    
    print("\nDifference between 100-year and 0.5-year profiles:")
    print(f"  Maximum difference: {wse_pivot['Diff_100yr_vs_05yr'].max():.2f} ft")
    print(f"  Minimum difference: {wse_pivot['Diff_100yr_vs_05yr'].min():.2f} ft")
    print(f"  Average difference: {wse_pivot['Diff_100yr_vs_05yr'].mean():.2f} ft")
    
    print("\nStations with largest differences:")
    top_diff = wse_pivot.nlargest(5, 'Diff_100yr_vs_05yr')[['100 year', '.5 year', 'Diff_100yr_vs_05yr']]
    print(top_diff)


Difference between 100-year and 0.5-year profiles:
  Maximum difference: 42.14 ft
  Minimum difference: 7.90 ft
  Average difference: 21.62 ft

Stations with largest differences:
Profile                        100 year     .5 year  Diff_100yr_vs_05yr
River      Reach   Station                                             
Bald Eagle Loc Hav 96370.43  662.169006  620.027832           42.141174
                   94560.01  662.168457  620.027466           42.140991
                   93391.71  662.167664  620.027283           42.140381
                   97607.35  662.168640  620.028320           42.140320
                   98206.87  662.169983  620.029846           42.140137


## Summary

This notebook demonstrated the new steady state functionality in ras-commander:

1. ✅ Checked if a plan contains steady state results
2. ✅ Extracted profile names for different return periods
3. ✅ Retrieved WSE data for individual and all profiles
4. ✅ Accessed steady flow metadata and attributes
5. ✅ Visualized water surface profiles
6. ✅ Analyzed differences between profiles

These tools enable comprehensive steady state flow analysis and comparison of hydraulic conditions across different design storms or flow scenarios.